In [1]:
import numpy as np

In [2]:
from past.builtins import xrange


In [3]:
from cs231n.classifiers.linear_svm import svm_loss_naive,svm_loss_vectorized

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
x = np.array([[6,6,8,1],[2,9,0,8],[2,5,4,8]])
w = np.array([[2,0],[3,0],[1,4],[3,1]])
dot = np.dot(x,w)
y_true = np.array([1,0,1])


In [6]:
S_true = dot[np.arange(3),np.squeeze(y_true)]

In [7]:
loss,drw = svm_loss_naive(w,x,y_true,0.05)

In [8]:
drw * 3

array([[  8.6,  -8. ],
       [ 11.9, -11. ],
       [ 12.3, -10.8],
       [  9.9,  -8.7]])

In [9]:
loss2,drw2 = svm_loss_vectorized(w,x,y_true,0.05)

In [10]:
drw2 * 3

array([[ 8.6, -2. ],
       [11.9, -5. ],
       [12.3, -2.8],
       [ 9.9, -7.7]])

In [11]:
drw - drw2

array([[ 0.        , -2.        ],
       [ 0.        , -2.        ],
       [ 0.        , -2.66666667],
       [ 0.        , -0.33333333]])

In [19]:
(dot.T - dot[np.arange(3),y_true]).T + 1

array([[  9,   1],
       [  1, -46],
       [ 24,   1]])

In [29]:
dW = np.zeros_like(w)
dW[:,1] -= (x[0] + x[2])
dW[:,0] += (x[0] + x[2])
print(dW)

[[  8  -8]
 [ 11 -11]
 [ 12 -12]
 [  9  -9]]


In [30]:
dW = dW.astype('float')

In [31]:
dW /= 3

In [32]:
dW

array([[ 2.66666667, -2.66666667],
       [ 3.66666667, -3.66666667],
       [ 4.        , -4.        ],
       [ 3.        , -3.        ]])

In [33]:
dW += (2 * w * 0.05)

In [34]:
dW

array([[ 2.86666667, -2.66666667],
       [ 3.96666667, -3.66666667],
       [ 4.1       , -3.6       ],
       [ 3.3       , -2.9       ]])

In [35]:
dW * 3

array([[  8.6,  -8. ],
       [ 11.9, -11. ],
       [ 12.3, -10.8],
       [  9.9,  -8.7]])

In [36]:
margins = np.maximum(0,(dot.T - dot[np.arange(3),y_true]).T + 1)

In [38]:
margins[range(3),y_true] = 0

In [39]:
margins

array([[ 9,  0],
       [ 0,  0],
       [24,  0]])

In [40]:
positive = margins > 0
positive

array([[ True, False],
       [False, False],
       [ True, False]])

In [66]:
dWT = np.zeros_like(w).T

In [77]:
positive =( margins > 0).astype(int)

In [78]:
np.dot(x.T, positive)

array([[ 8,  0],
       [11,  0],
       [12,  0],
       [ 9,  0]])

In [79]:
p2 = positive

In [80]:
p2[range(3),y_true] -= np.sum(positive,axis=1)

In [81]:
p2

array([[ 1, -1],
       [ 0,  0],
       [ 1, -1]])

In [82]:
np.dot(x.T,p2)

array([[  8,  -8],
       [ 11, -11],
       [ 12, -12],
       [  9,  -9]])